In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash
import dash_leaflet as dl

from dash import Dash, callback, html, dcc, dash_table, Input, Output, State, MATCH, ALL

#import dash_html_components as html
#import dash_core_components as dcc
#import dash_table
from dash.dash_table.Format import Group

#import dash_core_components as dcc
#import dash_html_components as html
from dash import dcc
from dash import html
import plotly.express as px
#import dash_table as dt
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuse"
password = "123qwe"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client        
        
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", 
        selected_rows=[],
        filter_action="native", 
        sort_action="native", 
        page_action="native", 
        page_current=0, 
        page_size=20, 

    ),
    html.Br(),
     html.Hr(),
     html.Div( #div for the map
            id='map-id',
            className='col s12 m6',
            )
    ])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#228B22'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(virtualRows):
    
    print(virtualRows)
    
#FIXME Add in the code for your geolocation chart

    #austin Texas is [30.75, -97.48]
    
    #create the views
    if not virtualRows: 
        markerArray = (30.75,-97.48) 
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: 
        dff = pd.DataFrame(df.iloc[virtualRows]) 
        coordLat = float(dff['location_lat'].to_string().split()[1]) 
        coordLong = float(dff['location_long'].to_string().split()[1]) 
        markerArray = (coordLat, coordLong) 
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    return [dl.Map(style={'width': '900px', 'height': '600px'}, center=[30.75,-97.48],
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]
    
app.run_server()

Connection Successful
Dash app running on http://127.0.0.1:25563/
None
[]
